In [8]:
from glob import glob
from src.systems.pendulum_cartesian import PendulumCartesianSystem
from src.flow_matching.pendulum_cartesian.latent_conditional.flow_matcher import PendulumCartesianLatentConditionalFlowMatcher
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import os
import torch

data_dir = "/common/users/shared/pracsys/genMoPlan/data_trajectories/pendulum_cartesian_50k"
roa_file = "/common/users/shared/pracsys/genMoPlan/data_trajectories/pendulum_cartesian_50k/roa_labels.txt"
bounds_file = "/common/users/dm1487/arcmg_datasets/pendulum_cartesian/pendulum_cartesian_data_bounds.pkl"


system = PendulumCartesianSystem(bounds_file=bounds_file, use_dynamic_bounds=True)

ckpt_path = "/common/home/dm1487/robotics_research/tripods/olympics-classifier/outputs/pendulum_cartesian_500/2025-11-11_01-39-41"
flow_matcher = PendulumCartesianLatentConditionalFlowMatcher.load_from_checkpoint(ckpt_path, device="cuda:0")

Loading Pendulum Cartesian bounds from: /common/users/dm1487/arcmg_datasets/pendulum_cartesian/pendulum_cartesian_data_bounds.pkl
Use dynamic bounds: True
Path exists: True
  [0] X Position: [-1.000, 1.000] -> limit: ±1.000
  [1] Y Position: [-1.000, 1.000] -> limit: ±1.000
  [2] X Velocity: [-6.282, 6.283] -> limit: ±6.283
  [3] Y Velocity: [-6.283, 6.283] -> limit: ±6.283
Loaded Pendulum Cartesian bounds from: /common/users/dm1487/arcmg_datasets/pendulum_cartesian/pendulum_cartesian_data_bounds.pkl
📁 Folder provided: /common/home/dm1487/robotics_research/tripods/olympics-classifier/outputs/pendulum_cartesian_500/2025-11-11_01-39-41
🔍 Searching for checkpoint in folder...
   ✓ Found best checkpoint (val_loss=0.0025)
   📄 Using: epoch1004-val_loss0.0025.ckpt
🤖 Loading Pendulum Cartesian LCFM checkpoint: /common/home/dm1487/robotics_research/tripods/olympics-classifier/outputs/pendulum_cartesian_500/2025-11-11_01-39-41/version_0/checkpoints/epoch1004-val_loss0.0025.ckpt
📍 Device: cuda:0

In [9]:
roa_data = np.loadtxt(roa_file, delimiter=",")
inp, labels = roa_data[:, :-1], roa_data[:, -1]
inp = torch.from_numpy(inp).float().to("cuda:0")
labels = torch.from_numpy(labels).long().to("cuda:0")
np.mean(roa_data[:, -1] == 1)


0.38585493269037574

In [10]:
from tqdm import tqdm

samples = 100
repeats = 1
batch_size = 2048  # You can tune this depending on memory
success_threshold = 0.6
failure_threshold = 0.4
tp = 0
tn = 0
fp = 0
fn = 0

sep_count = 0

start_idx = 0
stop_idx = len(roa_data)

is_success = np.zeros((len(roa_data), samples, repeats))
for batch_start in tqdm(range(start_idx, stop_idx, batch_size)):
    batch_end = min(batch_start + batch_size, stop_idx)
    batch_inp = inp[batch_start:batch_end, :]

    # Will be (batch_size, samples, repeats)
    for sample_idx in range(samples):
        model_input = batch_inp.clone()
        for repeat_idx in range(repeats):
            pred = flow_matcher.predict_endpoint(model_input)
            # pred shape: (batch_size, d)
            is_success[batch_start:batch_end, sample_idx, repeat_idx] = system.classify_attractor(pred, 0.1).cpu().numpy()
            # is_success[batch_start:batch_end, sample_idx, repeat_idx] = pred[:, 21].cpu().numpy() > 1.3
            model_input = pred.clone()
            

# is_success_mean = is_success.mean(axis=(1,2))
# pred_success = (is_success_mean > success_threshold)
# pred_failure = (is_success_mean < failure_threshold) 

# # Compute tp, tn, fp, fn
# batch_labels = labels[start_idx:stop_idx].cpu().numpy()
# tp = np.sum((batch_labels == 1) & pred_success)
# fp = np.sum((batch_labels == 0) & pred_success)
# fn = np.sum((batch_labels == 1) & pred_failure)
# tn = np.sum((batch_labels == 0) & pred_failure)
# sep_count = np.sum((is_success_mean <= success_threshold) & (is_success_mean >= failure_threshold))
#     # # Success logic: check along [samples, repeats] for each data point in batch
#     # # We'll take the mean across all samples and repeats for head height > 1.3
    
#     # mean_success = (all_head_heights > 1.3).mean(axis=(1,2))
#     # is_success = mean_success > success_threshold  # shape: (batch_size,)
    
#     # is_failure = mean_success < failure_threshold  # shape: (batch_size,)
    
#     # sep_count += np.sum((mean_success <= success_threshold) & (mean_success >= failure_threshold))

#     # batch_labels = labels[batch_start:batch_end].cpu().numpy()  # shape: (batch_size,)

#     # tp += np.sum((batch_labels == 1) & (is_success))
#     # fp += np.sum((batch_labels == 0) & (is_success))
#     # fn += np.sum((batch_labels == 1) & (is_failure))  
#     # tn += np.sum((batch_labels == 0) & (is_failure))



  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [01:31<00:00,  3.65s/it]


In [11]:
batch_labels = labels[start_idx:stop_idx].cpu().numpy()
pred_labels = np.ones_like(batch_labels) * -1

failure = (is_success == -1).sum(axis=(1, 2))/samples > 0.6
success = (is_success == 1).sum(axis=(1, 2))/samples > 0.6

pred_labels[failure] = 0
pred_labels[success] = 1

In [12]:
tp = np.sum((batch_labels == 1) & (pred_labels == 1))
tn = np.sum((batch_labels == 0) & (pred_labels == 0))
fp = np.sum((batch_labels == 0) & (pred_labels == 1))
fn = np.sum((batch_labels == 1) & (pred_labels == 0))


In [13]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
specificity = tn / (tn + fp)
sep_perc = sep_count/len(roa_data)
print(f"Precision: {precision}, Recall: {recall}, F1: {f1}, Specificity: {specificity}, Sep: {sep_perc}")

# confusion matrix
conf_mat = np.zeros((2, 2))
conf_mat[0, 0] = tp
conf_mat[0, 1] = fp
conf_mat[1, 0] = fn
conf_mat[1, 1] = tn


Precision: 0.9473771123465624, Recall: 0.9072268198829793, F1: 0.926867359577791, Specificity: 0.9683956494429568, Sep: 0.0


In [14]:
np.sum(pred_labels == -1) / len(roa_data)

0.011050833835643962